In [62]:
import pandas as pd
import psycopg2
import numpy as np


sqluser = 'mimicuser'
dbname = 'mimic'
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

q_departments = """SELECT count(*) from d_icd_diagnoses;"""

df_departments = pd.read_sql_query(q_departments, con)

In [4]:
df_departments

,count
0,14567


In [11]:
import pandas as pd
import psycopg2


sqluser = 'mimicuser'
dbname = 'mimic'
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

q_diagnoses = """SELECT count(*) from DIAGNOSES_ICD;"""

df_diagnoses = pd.read_sql_query(q_diagnoses, con)

In [13]:
df_diagnoses

,count
0,651047


In [14]:
import pandas as pd
import psycopg2


sqluser = 'mimicuser'
dbname = 'mimic'
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

q_merge = """SELECT d.SUBJECT_ID, d.HADM_ID, d.ICD9_CODE, lookup.SHORT_TITLE, lookup.LONG_TITLE 
                from DIAGNOSES_ICD d
                left join d_icd_diagnoses lookup 
                on d.ICD9_CODE = lookup.ICD9_CODE;"""

df_merge = pd.read_sql_query(q_merge, con)

In [15]:
len(df_merge)

651047

In [16]:
df_merge.head()

,subject_id,hadm_id,icd9_code,short_title,long_title
0,109,172335,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
1,109,172335,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified"
2,109,172335,58281,Chr nephritis in oth dis,Chronic glomerulonephritis in diseases classif...
3,109,172335,5855,Chron kidney dis stage V,"Chronic kidney disease, Stage V"
4,109,172335,4254,Prim cardiomyopathy NEC,Other primary cardiomyopathies


In [23]:
df_merge['icd9_code'].value_counts()

4019     20703
4280     13111
42731    12891
41401    12429
5849      9119
25000     9058
2724      8690
51881     7497
5990      6555
53081     6326
2720      5930
V053      5779
V290      5519
2859      5406
2449      4917
486       4839
2851      4552
2762      4528
496       4431
99592     3912
V5861     3806
0389      3725
5070      3680
V3000     3566
5859      3435
311       3431
40390     3421
3051      3358
412       3278
2875      3065
         ...  
45374        1
84512        1
37500        1
2161         1
37489        1
E8219        1
67204        1
9593         1
9949         1
37182        1
37612        1
1429         1
9125         1
36217        1
5262         1
6182         1
80146        1
8977         1
V4363        1
8363         1
38908        1
71515        1
75016        1
9048         1
0470         1
24991        1
2541         1
1308         1
E9499        1
32724        1
Name: icd9_code, Length: 6984, dtype: int64

In [30]:
# Reading the ICD 9 Codes. First three digit are important. Last two if available are supplementary
df_merge['ICD_PC'] = df_merge['icd9_code'].astype(str).str[:3]

In [33]:
df_merge.head()

,subject_id,hadm_id,icd9_code,short_title,long_title,ICD_PC
0,109,172335,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant...",403
1,109,172335,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",486
2,109,172335,58281,Chr nephritis in oth dis,Chronic glomerulonephritis in diseases classif...,582
3,109,172335,5855,Chron kidney dis stage V,"Chronic kidney disease, Stage V",585
4,109,172335,4254,Prim cardiomyopathy NEC,Other primary cardiomyopathies,425


In [47]:
#converting to numeric
df_merge['int_icd9']=pd.to_numeric(df_merge.ICD_PC, errors='coerce')

In [112]:
conditions = [
    (df_merge['int_icd9'] > 0) & (df_merge['int_icd9'] <= 139),
    (df_merge['int_icd9'] >= 140) & (df_merge['int_icd9'] <= 239),
    (df_merge['int_icd9'] >= 240) & (df_merge['int_icd9'] <= 279),
    (df_merge['int_icd9'] >= 280) & (df_merge['int_icd9'] <= 289),
    (df_merge['int_icd9'] >= 290) & (df_merge['int_icd9'] <= 319),
    (df_merge['int_icd9'] >= 320) & (df_merge['int_icd9'] <= 389),
    (df_merge['int_icd9'] >= 390) & (df_merge['int_icd9'] <= 459),
    (df_merge['int_icd9'] >= 460) & (df_merge['int_icd9'] <= 519),
    (df_merge['int_icd9'] >= 520) & (df_merge['int_icd9'] <= 579),
    (df_merge['int_icd9'] >= 580) & (df_merge['int_icd9'] <= 629),
    (df_merge['int_icd9'] >= 630) & (df_merge['int_icd9'] <= 679),
    (df_merge['int_icd9'] >= 680) & (df_merge['int_icd9'] <= 709),
    (df_merge['int_icd9'] >= 710) & (df_merge['int_icd9'] <= 739),
    (df_merge['int_icd9'] >= 740) & (df_merge['int_icd9'] <= 759),
    (df_merge['int_icd9'] >= 760) & (df_merge['int_icd9'] <= 779),
    (df_merge['int_icd9'] >= 780) & (df_merge['int_icd9'] <= 799),
    (df_merge['int_icd9'] >= 800) & (df_merge['int_icd9'] <= 999),
    (df_merge.icd9_code.str.startswith('E', na=False)) | (df_merge.icd9_code.str.startswith('V', na=False)),
    
]
choices = ['infectious and parasitic diseases', 'neoplasms',\
           'endocrine, nutritional and metabolic diseases, and immunity disorders',\
           'diseases of the blood and blood-forming organs','mental disorders',\
           'diseases of the nervous system and sense organs','diseases of the circulatory system',\
           'diseases of the respiratory system','diseases of the digestive system',\
           'diseases of the genitourinary system','complications of pregnancy, childbirth, and the puerperium',\
           'diseases of the skin and subcutaneous tissue','diseases of the musculoskeletal system and connective tissue',\
          'congenital anomalies','certain conditions originating in the perinatal period','symptoms, signs, and ill-defined conditions',\
          'injury and poisoning','external causes of injury and supplemental classification']
df_merge['Department'] = np.select(conditions, choices, default='null')
#print(df_merge)

In [113]:
df_merge.head()

,subject_id,hadm_id,icd9_code,short_title,long_title,int_icd9,Department
0,109,172335,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant...",403.0,diseases of the circulatory system
1,109,172335,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",486.0,diseases of the respiratory system
2,109,172335,58281,Chr nephritis in oth dis,Chronic glomerulonephritis in diseases classif...,582.0,diseases of the genitourinary system
3,109,172335,5855,Chron kidney dis stage V,"Chronic kidney disease, Stage V",585.0,diseases of the genitourinary system
4,109,172335,4254,Prim cardiomyopathy NEC,Other primary cardiomyopathies,425.0,diseases of the circulatory system


In [114]:
#df_merge = df_merge.drop(['ICD_PC'],axis=1)

In [115]:
df_merge.Department.value_counts()

diseases of the circulatory system                                       140257
external causes of injury and supplemental classification                 97249
endocrine, nutritional and metabolic diseases, and immunity disorders     69592
diseases of the respiratory system                                        44825
injury and poisoning                                                      42948
diseases of the digestive system                                          38527
diseases of the genitourinary system                                      32867
symptoms, signs, and ill-defined conditions                               29691
diseases of the blood and blood-forming organs                            25262
mental disorders                                                          24876
diseases of the nervous system and sense organs                           23334
infectious and parasitic diseases                                         20264
certain conditions originating in the pe

In [116]:
df_null = df_merge[df_merge['Department']=='null']

In [119]:
#ICD codes are null
df_null.head()

,subject_id,hadm_id,icd9_code,short_title,long_title,int_icd9,Department
5251,417,102633,None,None,None,NaN,null
8354,690,174817,None,None,None,NaN,null
9691,937,177274,None,None,None,NaN,null
18667,1490,135580,None,None,None,NaN,null
27313,3369,126808,None,None,None,NaN,null
